# 資料整理

In [25]:

import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import torch

datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    'C:\\Users\\walter\\OneDrive\\桌面\\收集\\大學nn專題\\nn實作\\阿茲海默症預測\\archive\\Dataset',
    target_size=(128, 128),#保持
    batch_size=32,
    class_mode='binary',
    shuffle=True)

# Split the data into training and testing sets
data, labels = next(data_generator)
for _ in range(len(data_generator) - 1):
    imgs, lbls = next(data_generator)
    data = np.append(data, imgs, axis=0)
    labels = np.append(labels, lbls, axis=0)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

train_data = torch.tensor(train_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_data = torch.tensor(test_data, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)
train_data = train_data.permute(0, 3, 1, 2)  # Change data shape from (batch, height, width, channels) to (batch, channels, height, width)
test_data = test_data.permute(0, 3, 1, 2)  # Change data shape from (batch, height, width, channels) to (batch, channels, height, width)

# 獲取所有唯一的label
unique_labels = torch.unique(test_labels)

# 創建一個字典來保存分類後的數據
grouped_test_data = {label.item(): [] for label in unique_labels}

# 遍歷所有label，將對應的data加入字典中的相應列表
for label in unique_labels:
    grouped_test_data[label.item()] = test_data[test_labels == label]
    print(f"Label {label.item()}: {grouped_test_data[label.item()].size()}")
label_to_name = {  
    0: 'Non_Demented',  
    1: 'Very_Mild_Demented',  
    2: 'Mild_Demented',  
    3: 'Moderate_Demented'  
}

# grouped_data 現在包含了分類後的數據

Found 6400 images belonging to 4 classes.
Label 0: torch.Size([175, 3, 128, 128])
Label 1: torch.Size([15, 3, 128, 128])
Label 2: torch.Size([620, 3, 128, 128])
Label 3: torch.Size([470, 3, 128, 128])


# 模型

In [57]:
BATCH_SIZE = 128
CHANNELS = 3
IMG_DIM = 128
NGPU = 1
device = torch.device("cuda:0")

from torch import nn,optim
import time
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight


device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class LeNet(nn.Module):
    def __init__(self) :
        super(LeNet,self).__init__()
        self.conv=nn.Sequential(
            #in_channels ,out_channels, kernel_size
            nn.Conv2d(CHANNELS,3,5),
            nn.Tanh(),
            #kernel_size, stride
            nn.MaxPool2d(2,2),
            nn.Conv2d(3,3,5),
            nn.Tanh(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(3,3,6),
            nn.Tanh(),
            nn.MaxPool2d(2,2)

        )
        self.dropout = nn.Dropout(p=0.5)

        self.fc=nn.Sequential(
            nn.Linear(3*12*12,360),
            nn.Tanh(),
            nn.Linear(360,120),
            nn.Tanh(),
            nn.Linear(120,4),
            nn.Tanh()
        )
    def forward(self,img):
        feature=self.conv(img)
       # print("Conv output shape: ", feature.shape)  # 打印卷積層輸出的形狀
        
        output=self.fc(feature.view(img.shape[0],-1))
        output=self.dropout(output)
        return output


def evaluate_accuracy(data_iter,net,device=None):
    if device is None and isinstance(net,torch.nn.Module):
        device=list(net.parameters())[0].device
    acc_sum,n=0.0,0
    for x,y in data_iter:
        if isinstance(net,torch.nn.Module):
            #評估模式，關閉dropout
            net.eval()
            x=x.to(device)
            y=y.to(device)
            acc_sum+=(net(x).argmax(dim=1)==y).float().sum().cpu().item()
            net.train()
        else:
            if('is_training' in net.__code__.co_varnames):
                acc_sum+=(net(x,is_training=False).argmax(dim=1)==y).float().sum().cpu().item()
            else:
                acc_sum+=(net(x).argmax(dim=1)==y).float().sum().cpu().item()
        n+=y.shape[0]
    return acc_sum/n

def train(net,train_data,train_labels,test_data,test_labels,batch_size,optimizer,device,num_epochs):
    
    # 學習率調度器
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.99, verbose=True)

    #train_iter-測試樣本劃分為最小批的結果
    # 初始化 accuracy_dict，為每個label名稱創建一個空列表
    accuracy_dict = {name: [] for name in label_to_name.values()}
    ls=[]
    net=net.to(device)
    print("training on ",device)

    a=1/grouped_test_data[0].size(0)
    b=1/grouped_test_data[1].size(0)
    c=1/grouped_test_data[2].size(0)
    d=1/grouped_test_data[3].size(0)
    weights = torch.tensor([a/a+b+c+d, b/a+b+c+d, c/a+b+c+d,d/a+b+c+d]).to('cuda')   # 這裡的數字可以根據需要調整

  # 這裡的數字可以根據需要調整
    print("weights",weights)

    loss = torch.nn.CrossEntropyLoss(weight=weights)
  
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n,batch_count=0.0,0.0,0,0
        start=time.time()
        dataset=torch.utils.data.TensorDataset(train_data,train_labels)#將標籤和特徵打包
        #DataLoader本質是一迭代器，批次處理data
        train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)#shuffle=True為打亂dataset

        for data,labels in train_iter:
            data=data.to(device)
            labels=labels.to(device)

            y_hat=net(data)
            l=loss(y_hat,labels)
            #梯度清零
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            ls.append(l)
            train_l_sum+=l.cpu().item()
            train_acc_sum+=(y_hat.argmax(dim=1)==labels).sum().cpu().item()
            n+=labels.shape[0]
            batch_count+=1
        
        
        print('epoch ',epoch+1,',loss ',train_l_sum/batch_count,', train acc',train_acc_sum/n)
        # 遍歷每個label
        for label in grouped_test_data.keys():
            # 獲取對應的 data 和 label
            data = grouped_test_data[label]
            labels = torch.tensor([label] * len(data), dtype=torch.long)  # 為每個樣本生成相應的 label

            # 創建 TensorDataset 和 DataLoader
            dataset = TensorDataset(data, labels)
            data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            # 計算該label的準確度
            test_acc = evaluate_accuracy(data_loader, net)
            # 使用label_to_name將label轉換為名稱，並將test_acc儲存到字典中
            name = label_to_name[label]
            accuracy_dict[name].append(test_acc)
            print('label',name,"acc",test_acc)

        print('time',time.time()-start,'sec','net par')
        #scheduler.step(test_acc)

    
        ''' 
        for name, param in net.conv.named_parameters():
            print(f'{name} grad: {param.grad}')
        '''

    x=np.linspace(0,num_epochs,num=num_epochs)
    for label in grouped_test_data.keys():
        name = label_to_name[label]  # 将label转换为对应的名称
        plt.plot(x, accuracy_dict[name], label=name)    
    plt.xlabel('epoch')
    plt.ylabel('test_acc')
    plt.legend()
    plt.show()

    x_2=np.linspace(0,num_epochs,len(ls))
    plt.plot(x_2,ls)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

from torch.nn import init


num_epochs=500
net=LeNet()
# 初始化网络参数
for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)
toptimizer = optim.Adam(net.parameters(), lr=0.0005)#weight0.0005 原:0.001
train(net,train_data=train_data,train_labels=train_labels,test_data=test_data,test_labels=test_labels,batch_size=40,optimizer=toptimizer,device='cuda',num_epochs=num_epochs)
print("t7")

torch.cuda.empty_cache()


training on  cuda
weights tensor([ 1.0704, 11.7371,  0.3527,  0.4427], device='cuda:0')


KeyboardInterrupt: 